In [ ]:
import numpy as np
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt
import cmocean as cmo
import pandas as pd
import matplotlib.animation as animation
from matplotlib.colors import LightSource
from matplotlib import cm
import matplotlib.colors
from matplotlib.gridspec import GridSpec

# Import colormaps and limits
import sys
sys.path.append('./')
from colormaps import *

In [2]:
data_dir = '../../Zenodo/model_output'

# QU_M, PI_M, PL_M
M_QUN = xr.open_dataset(f'{data_dir}/QU_M.nc')
M_PIN = xr.open_dataset(f'{data_dir}/PI_M.nc')
M_PLN = xr.open_dataset(f'{data_dir}/PL_M.nc')

# QU_H, PI_H, PL_H
H_QUN = xr.open_dataset(f'{data_dir}/QU_H.nc')
H_PIN = xr.open_dataset(f'{data_dir}/PI_H.nc')
H_PLN = xr.open_dataset(f'{data_dir}/PL_H.nc')

spinup = xr.open_dataset(f'{data_dir}/spinup.nc')  


In [3]:
labels = ['LADDIE', 'QU_M', 'PI_M', 'PL_M']
colors = ['C3', 'C9', 'C8', 'C7']

files_M             = [M_QUN, M_PIN, M_PLN]
files_H             = [H_QUN, H_PIN, H_PLN]

In [ ]:
files       = files_M
labels      = ['LADDIE', 'QU_M', 'PI_M', 'PL_M']
xslicenew   = [ -100000, -250000]
times       = [200,1000]

# Create figure
fig = plt.figure(figsize=(16,10), dpi=300)
gs = fig.add_gridspec(4, 11, height_ratios=[(250000-xslicenew[0])/270000, (250000-xslicenew[1])/270000, .2, .2],
              width_ratios=[1, 1, 1, .2, 1, 1, 1,.2, 1, 1, 1], hspace=0.1, wspace=0.15)

# Add colorbars
caxHi = fig.add_subplot(gs[3, 0:3])
caxme = fig.add_subplot(gs[3, 4:7])
caxi = fig.add_subplot(gs[3, 8:11])

panel_labels = np.array([['(a)', '(b)', '(c)'],['(d)', '(e)', '(f)'],['(g)', '(h)', '(i)'],['(j)', '(k)', '(l)'],['(m)', '(n)', '(o)'],['(p)', '(q)', '(r)']])


# Loop over files
for col, file in enumerate(files):

    # Loop over time slices, don't plot for t = 0
    for row, t in enumerate(times):
        
        # Select time slice and xslice
        ds = file.sel(time=t, x = slice(xslicenew[row],250000))

        # Transpose data
        newx = ds.y[::-1]/1000
        newy = ds.x/1000
        
        newHi =  ds.Hs.T - ds.Hi_eff_cf.T
        newBMB = ds.BMB.T*-1
        newmask = ds.mask.T
        newfground = ds.f_grnd.T
        
        # u_surf and v_surf are not written to t = 0, so take next time step (FJFJ: fix later, take field from help fields spinup)
        if t==0: 
            dsIM0 = file.sel(time=t+.125, x = slice(xslicenew[row],250000))
            newicevelu = -dsIM0['v_surf'].T
            newicevelv = dsIM0['u_surf'].T
            newfground = dsIM0['f_grnd'].T
        else:
            newicevelu = -ds['v_surf'].T
            newicevelv = ds['u_surf'].T
            newfground = ds['f_grnd'].T
        newmaskgl = xr.where(ds['mask']==7, 1,0).T

        # Compute deep melt average [m/yr] and integrated melt flux [Gt/yr]
        deepmelt    = np.nanmean((xr.where(np.logical_and(ds.mask==4, ds.Hs - ds.Hi < -300), 1, np.nan) * - ds.BMB).T)
        intmelt     = np.nansum(newBMB * 2000 * 2000 * 918 * 1e-12) # melt in m ice per year * area per gridcell * rhoice * 1e-12 [conversion kg to Gt]

        # Plot draft
        axHi = fig.add_subplot(gs[row, col*4])
        imHi = axHi.pcolormesh(newx, newy, newHi, cmap=cmap_draft, norm = norm_draft)
        # Add contours
        # CD = axHi.contour(newx, newy, newHi,  [-300, -275, -250, -225, -200, -175, -150, -125, -100], cmap='inferno_r',zorder=1)
        axHi.contour(newx, newy, -newHi,  [100, 125, 150, 175, 200, 225, 250, 300], colors='black', linewidths=3, zorder=1)
        CD = axHi.contour(newx, newy, -newHi,  [100, 125, 150, 175, 200, 225, 250, 300], cmap='cmo.deep',  zorder=1)
        h, l = CD.legend_elements()

        # Mask grounded ice and ocean
        axHi.pcolormesh(newx, newy,    xr.where(np.logical_or(newmask==3,newmask==7), 1, np.nan),   cmap=cmap_ground,   norm=norm_ground)
        axHi.pcolormesh(newx, newy,    xr.where(newmask==1, 1, np.nan),                             cmap=cmap_ocean,    norm=norm_ocean)    
        


        # Plot melt
        axme = fig.add_subplot(gs[row, 1+col*4])
        immelt = axme.pcolormesh(newx, newy, newBMB, cmap=cmap_BMB, norm=norm_BMB)
        # Print deepmelt and intmelt
        axme.text(38, xslicenew[row]/1000+25, f'{deepmelt:.2f} m/yr', fontsize=11, horizontalalignment='right')
        axme.text(38, xslicenew[row]/1000+5, f'{intmelt:.2f} Gt/yr', fontsize=11,horizontalalignment='right')
        # Mask grounded ice and ocean
        axme.pcolormesh(newx, newy,    xr.where(np.logical_or(newmask==3,newmask==7), 1, np.nan),   cmap=cmap_ground,   norm=norm_ground)        
        axme.pcolormesh(newx, newy,    xr.where(newmask==1, 1, np.nan),                             cmap=cmap_ocean,    norm=norm_ocean) 

        # Plot ice velocity
        axi = fig.add_subplot(gs[row, 2+col*4])
        imi = axi.pcolormesh( newx, newy, np.sqrt(newicevelu**2+newicevelv**2), cmap=cmap_u_surf, norm = norm_u_surf)
        # Mask grounding line and ocean
        axi.pcolormesh(newx, newy,    xr.where(np.logical_and(newfground<1, newfground>0), 1, np.nan),   cmap=cmap_GL,      norm=norm_GL)
        axi.pcolormesh(newx, newy,    xr.where(newmask==1, 1, np.nan), cmap=cmap_ocean,    norm=norm_ocean) 
        # Show velocity vectors        
        stepx = 5   # x spacing vectors
        stepy = 40  # y spacing vectors
        axi.quiver(newx[::stepx], newy[::stepy], newicevelu[::stepy,::stepx], newicevelv[::stepy, ::stepx], scale=4000, width=1/100, facecolor='white')

        # Set labels and titles
        plt.setp( axme.get_yticklabels(), visible=False)
        plt.setp( axi.get_yticklabels(), visible=False)

        axHi.set_aspect(0.5)
        axme.set_aspect(0.5)
        axi.set_aspect(0.5)

        axHi.set_xticks([-40,0,40])
        axme.set_xticks([-40,0,40])
        axi.set_xticks([-40,0,40])

        if row==1:
            axHi.set_xlabel('[km]', fontsize=14)
            axme.set_xlabel('[km]', fontsize=14)
            axi.set_xlabel('[km]', fontsize=14)
            axHi.set_xticklabels([-40,0,40],rotation=45)
            axme.set_xticklabels([-40,0,40],rotation=45)
            axi.set_xticklabels([-40,0,40],rotation=45)
            axHi.text(-39,-240,  panel_labels[col+3, 0],weight='bold')
            axme.text(-39,-240,  panel_labels[col+3, 1],weight='bold')
            axi.text(-39,-240,   panel_labels[col+3, 2],color='white',weight='bold')
        else:
            plt.setp( axHi.get_xticklabels(), visible=False)
            plt.setp( axme.get_xticklabels(), visible=False)
            plt.setp( axi.get_xticklabels(), visible=False)
            axHi.text(-39,-90,  panel_labels[col, 0],weight='bold')
            axme.text(-39,-90,  panel_labels[col, 1],weight='bold')
            axi.text(-39,-90,   panel_labels[col, 2],color='white',weight='bold')

        if col==0:
            axHi.set_ylabel('[km]', fontsize=14)
            axHi.set_yticks(newy[::-25])
            axHi.text(-120, (xslicenew[row]+100000)/2000, f'Model year {t}', fontsize=14, rotation=90)
        else:
            plt.setp( axHi.get_yticklabels(), visible=False)
            

        # Add title experiments
        if row == 0:
            axme.set_title(labels[col+1], color=colors[col+1], fontsize=16, weight='bold', ha='center')

        # Add legend draft contours
        if col ==0 and  row == 1:
            axHi.legend(h,['-100 m', '-125 m', '-150 m', '-175 m', '-200 m', '-225 m', '-250 m', '-300 m'],bbox_to_anchor=(-1.6, .2),loc=2)

        # Add EM and WM markers
        if row==0:
            if col==0:
                axHi.text(-35  , 43,r'GL'   ,size=8, weight='bold', ha='center', va='center', rotation=0)
                axHi.text(-31  , 35,r'w'    ,size=5, weight='bold', ha='center', va='center', rotation=0)
                axHi.text( 34  , 43,r'GL'   ,size=8, weight='bold', ha='center', va='center', rotation=0)
                axHi.text( 38  , 35,r'e'    ,size=5, weight='bold', ha='center', va='center', rotation=0)


# Set colorbars
cbar = plt.colorbar(imHi, ax = caxHi, ticks=[-500, -250, 0], label='Ice draft [m]', extend='min', orientation='horizontal', pad=.1, fraction=1)
cbar.ax.set_xticklabels([-500, -250, 0])
caxHi.set_axis_off()
cbar.set_label('Ice draft [m]', fontsize=14)
cbar.ax.tick_params(labelsize=14)

cbar = plt.colorbar(immelt, ax = caxme, ticks=[-10,-1, 0, 1, 10, 100], label='Sub-shelf melt [m/yr]', extend='both', orientation='horizontal', pad=.1, fraction=1)
cbar.ax.set_xticklabels([-10,-1, 0, 1, 10, 100])
caxme.set_axis_off()
cbar.set_label('Sub-shelf melt [m/yr]', fontsize=14)
cbar.ax.tick_params(labelsize=14)

cbar = plt.colorbar(imi, ax = caxi, ticks = [0,1000,2000],label='Ice velocity [m/yr]', extend='max', orientation='horizontal', pad=.1, fraction=1)
cbar.ax.set_xticklabels([0, 1000, 2000])
caxi.set_axis_off()
cbar.set_label('Ice velocity [m/yr]', fontsize=14)
cbar.ax.tick_params(labelsize=14)

# Save fig
fig.savefig('fig06.pdf', dpi=300, bbox_inches='tight')

In [ ]:
files       = files_H
labels      = ['LADDIE', 'QU_H', 'PI_H', 'PL_H']
xslicenew   = [ -100000, -250000]
times       = [200,1000]

# Create figure
fig = plt.figure(figsize=(16,10), dpi=300)
gs = fig.add_gridspec(4, 11, height_ratios=[(250000-xslicenew[0])/270000, (250000-xslicenew[1])/270000, .2, .2],
              width_ratios=[1, 1, 1, .2, 1, 1, 1,.2, 1, 1, 1], hspace=0.1, wspace=0.15)

# Add colorbars
caxHi = fig.add_subplot(gs[3, 0:3])
caxme = fig.add_subplot(gs[3, 4:7])
caxi = fig.add_subplot(gs[3, 8:11])

panel_labels = np.array([['(a)', '(b)', '(c)'],['(d)', '(e)', '(f)'],['(g)', '(h)', '(i)'],['(j)', '(k)', '(l)'],['(m)', '(n)', '(o)'],['(p)', '(q)', '(r)']])


# Loop over files
for col, file in enumerate(files):

    # Loop over time slices, don't plot for t = 0
    for row, t in enumerate(times):
        
        # Select time slice and xslice
        ds = file.sel(time=t, x = slice(xslicenew[row],250000))

        # Transpose data
        newx = ds.y[::-1]/1000
        newy = ds.x/1000
        
        newHi =  ds.Hs.T - ds.Hi_eff_cf.T
        newBMB = ds.BMB.T*-1
        newmask = ds.mask.T
        newfground = ds.f_grnd.T
        
        # u_surf and v_surf are not written to t = 0, so take next time step (FJFJ: fix later, take field from help fields spinup)
        if t==0: 
            dsIM0 = file.sel(time=t+.125, x = slice(xslicenew[row],250000))
            newicevelu = -dsIM0['v_surf'].T
            newicevelv = dsIM0['u_surf'].T
            newfground = dsIM0['f_grnd'].T
        else:
            newicevelu = -ds['v_surf'].T
            newicevelv = ds['u_surf'].T
            newfground = ds['f_grnd'].T
        newmaskgl = xr.where(ds['mask']==7, 1,0).T

        # Compute deep melt average [m/yr] and integrated melt flux [Gt/yr]
        deepmelt    = np.nanmean((xr.where(np.logical_and(ds.mask==4, ds.Hs - ds.Hi < -300), 1, np.nan) * - ds.BMB).T)
        intmelt     = np.nansum(newBMB * 2000 * 2000 * 918 * 1e-12) # melt in m ice per year * area per gridcell * rhoice * 1e-12 [conversion kg to Gt]

        # Plot draft
        axHi = fig.add_subplot(gs[row, col*4])
        imHi = axHi.pcolormesh(newx, newy, newHi, cmap=cmap_draft, norm = norm_draft)
        # Add contours
        # CD = axHi.contour(newx, newy, newHi,  [-300, -275, -250, -225, -200, -175, -150, -125, -100], cmap='inferno_r',zorder=1)
        axHi.contour(newx, newy, -newHi,  [100, 125, 150, 175, 200, 225, 250, 300], colors='black', linewidths=3, zorder=1)
        CD = axHi.contour(newx, newy, -newHi,  [100, 125, 150, 175, 200, 225, 250, 300], cmap='cmo.deep',  zorder=1)
        h, l = CD.legend_elements()

        # Mask grounded ice and ocean
        axHi.pcolormesh(newx, newy,    xr.where(np.logical_or(newmask==3,newmask==7), 1, np.nan),   cmap=cmap_ground,   norm=norm_ground)
        axHi.pcolormesh(newx, newy,    xr.where(newmask==1, 1, np.nan),                             cmap=cmap_ocean,    norm=norm_ocean)    
        


        # Plot melt
        axme = fig.add_subplot(gs[row, 1+col*4])
        immelt = axme.pcolormesh(newx, newy, newBMB, cmap=cmap_BMB, norm=norm_BMB)
        # Print deepmelt and intmelt
        axme.text(38, xslicenew[row]/1000+25, f'{deepmelt:.2f} m/yr', fontsize=11, horizontalalignment='right')
        axme.text(38, xslicenew[row]/1000+5, f'{intmelt:.2f} Gt/yr', fontsize=11,horizontalalignment='right')
        # Mask grounded ice and ocean
        axme.pcolormesh(newx, newy,    xr.where(np.logical_or(newmask==3,newmask==7), 1, np.nan),   cmap=cmap_ground,   norm=norm_ground)        
        axme.pcolormesh(newx, newy,    xr.where(newmask==1, 1, np.nan),                             cmap=cmap_ocean,    norm=norm_ocean) 

        # Plot ice velocity
        axi = fig.add_subplot(gs[row, 2+col*4])
        imi = axi.pcolormesh( newx, newy, np.sqrt(newicevelu**2+newicevelv**2), cmap=cmap_u_surf, norm = norm_u_surf)
        # Mask grounding line and ocean
        axi.pcolormesh(newx, newy,    xr.where(np.logical_and(newfground<1, newfground>0), 1, np.nan),   cmap=cmap_GL,      norm=norm_GL)
        axi.pcolormesh(newx, newy,    xr.where(newmask==1, 1, np.nan), cmap=cmap_ocean,    norm=norm_ocean) 
        # Show velocity vectors        
        stepx = 5   # x spacing vectors
        stepy = 40  # y spacing vectors
        axi.quiver(newx[::stepx], newy[::stepy], newicevelu[::stepy,::stepx], newicevelv[::stepy, ::stepx], scale=4000, width=1/100, facecolor='white')

        # Set labels and titles
        plt.setp( axme.get_yticklabels(), visible=False)
        plt.setp( axi.get_yticklabels(), visible=False)

        axHi.set_aspect(0.5)
        axme.set_aspect(0.5)
        axi.set_aspect(0.5)

        axHi.set_xticks([-40,0,40])
        axme.set_xticks([-40,0,40])
        axi.set_xticks([-40,0,40])

        if row==1:
            axHi.set_xlabel('[km]', fontsize=14)
            axme.set_xlabel('[km]', fontsize=14)
            axi.set_xlabel('[km]', fontsize=14)
            axHi.set_xticklabels([-40,0,40],rotation=45)
            axme.set_xticklabels([-40,0,40],rotation=45)
            axi.set_xticklabels([-40,0,40],rotation=45)
            axHi.text(-39,-240,  panel_labels[col+3, 0],weight='bold')
            axme.text(-39,-240,  panel_labels[col+3, 1],weight='bold')
            axi.text(-39,-240,   panel_labels[col+3, 2],color='white',weight='bold')
        else:
            plt.setp( axHi.get_xticklabels(), visible=False)
            plt.setp( axme.get_xticklabels(), visible=False)
            plt.setp( axi.get_xticklabels(), visible=False)
            axHi.text(-39,-90,  panel_labels[col, 0],weight='bold')
            axme.text(-39,-90,  panel_labels[col, 1],weight='bold')
            axi.text(-39,-90,   panel_labels[col, 2],color='white',weight='bold')

        if col==0:
            axHi.set_ylabel('[km]', fontsize=14)
            axHi.set_yticks(newy[::-25])
            axHi.text(-120, (xslicenew[row]+100000)/2000, f'Model year {t}', fontsize=14, rotation=90)
        else:
            plt.setp( axHi.get_yticklabels(), visible=False)
            

        # Add title experiments
        if row == 0:
            axme.set_title(labels[col+1], color=colors[col+1], fontsize=16, weight='bold', ha='center')

        # Add legend draft contours
        if col ==0 and  row == 1:
            axHi.legend(h,['-100 m', '-125 m', '-150 m', '-175 m', '-200 m', '-225 m', '-250 m', '-300 m'],bbox_to_anchor=(-1.6, .2),loc=2)

        # Add EM and WM markers
        if row==0:
            if col==0:
                axHi.text(-35  , 43,r'GL'   ,size=8, weight='bold', ha='center', va='center', rotation=0)
                axHi.text(-31  , 35,r'w'    ,size=5, weight='bold', ha='center', va='center', rotation=0)
                axHi.text( 34  , 43,r'GL'   ,size=8, weight='bold', ha='center', va='center', rotation=0)
                axHi.text( 38  , 35,r'e'    ,size=5, weight='bold', ha='center', va='center', rotation=0)

# Set colorbars
cbar = plt.colorbar(imHi, ax = caxHi, ticks=[-500, -250, 0], label='Ice draft [m]', extend='min', orientation='horizontal', pad=.1, fraction=1)
cbar.ax.set_xticklabels([-500, -250, 0])
caxHi.set_axis_off()
cbar.set_label('Ice draft [m]', fontsize=14)
cbar.ax.tick_params(labelsize=14)

cbar = plt.colorbar(immelt, ax = caxme, ticks=[-10,-1, 0, 1, 10, 100], label='Sub-shelf melt [m/yr]', extend='both', orientation='horizontal', pad=.1, fraction=1)
cbar.ax.set_xticklabels([-10,-1, 0, 1, 10, 100])
caxme.set_axis_off()
cbar.set_label('Sub-shelf melt [m/yr]', fontsize=14)
cbar.ax.tick_params(labelsize=14)

cbar = plt.colorbar(imi, ax = caxi, ticks = [0,1000,2000],label='Ice velocity [m/yr]', extend='max', orientation='horizontal', pad=.1, fraction=1)
cbar.ax.set_xticklabels([0, 1000, 2000])
caxi.set_axis_off()
cbar.set_label('Ice velocity [m/yr]', fontsize=14)
cbar.ax.tick_params(labelsize=14)

# Save fig
fig.savefig('figC2.pdf', dpi=300, bbox_inches='tight')